In [ ]:
#importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pylab import rcParams
from statsmodels.tsa.stattools import adfuller
from fbprophet import Prophet
from sklearn import metrics

In [ ]:
link = 'https://github.com/HamoyeHQ/HDSC-Time-series-analysis-and-forecast/files/7768846/Time_series_analysis_and_forecast_DATASET.csv'
data = pd.read_csv(link, encoding = 'latin-1')
data.head()

In [ ]:
# Converting FullDate column into Date time
data['FullDate'] = pd.to_datetime(data['FullDate'])
data.describe()

In [ ]:
data.set_index('FullDate', inplace = True) #FullDate as an index

In [ ]:
plt.plot(data.index, data.ElecPrice, '--', marker='*')
plt.figure(figsize=(20,8)) 
plt.grid()
plt.xlabel('Year')
plt.ylabel('Electricity Price')
plt.title('Electricity Price over a time')

In [ ]:
#Plot of ElecPrice over time
quarter_rate = data.resample('3M').sum()
daily_rate = data.resample('D').sum()
plt.figure(figsize= (20,7) )
plt.plot(daily_rate.index, daily_rate.ElecPrice, '--', marker='*')
plt.grid()
plt.xlabel('Year')
plt.ylabel('Electricity Price')
plt.title('Electricity Price over a time (sampled Data)')

In [ ]:
# Information of ADF test
def display_adfuller_info(adfull):
  print(f'ADF Statistic: {adfull[0]}')
  print(f'p-value: {adfull[1]}')
  print(f'No. of lags used: {adfull[2]}')
  print(f'No. of observations used : {adfull[3]}')
  print('Critical Values:')
  for k, v in adfull[4].items():
      print(f' {k}: {v}')
    sys_load = adfuller(daily_rate['SysLoad'])
display_adfuller_info(sys_load)

In [ ]:
gas_price = adfuller(daily_rate['GasPrice'])
display_adfuller_info(gas_price)

In [ ]:
elec_price = adfuller(daily_rate['ElecPrice'])
display_adfuller_info(elec_price)

In [ ]:
round(sys_load[1],6)
round(gas_price[1],6)
round(gas_price[4]["10%"],4)
round(elec_price[4]["5%"],4)

In [ ]:
# Splitting daily sampled rate into train and test 
train_data = daily_rate.iloc[:2757,:]
test_data = daily_rate.iloc[2757:,:]
train_data.shape , test_data.shape

In [ ]:
# Reset the index
train_data.reset_index(inplace = True)
train_data.head()

In [ ]:
# Rename the columns
df_train = train_data.iloc[:,[0,-1]]
df_train = df_train.rename(columns={"FullDate": "ds", "ElecPrice": "y"})
df_train.head()

In [ ]:
# Reset the index
test_data.reset_index(inplace = True)
test_data.head()

In [ ]:
# Rename the required columns
test_df = test_data.iloc[:,[0,-1]]
test_df = test_df.rename(columns={"FullDate": "ds", "ElecPrice": "y"})
test_df.head()

In [ ]:
# Model Implementation
model = Prophet()
model.fit(df_train)
forecast_result = model.predict(test_df) 
forecast_result.head()

In [ ]:
#MAPE
def Metric(y_true,y_pred):
  y_true,y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred)/y_true)) *100
MAPE = Metric(test_df['y'],forecast_result['yhat'])
round(MAPE,2)

In [ ]:
RMSE = np.sqrt(metrics.mean_squared_error(test_df['y'],forecast_result['yhat']))
round(RMSE,2)

In [ ]:
model.plot_components(forecast_result)

In [ ]:
# Reset the index
daily_rate.reset_index(inplace=True)
daily_rate.head(2)

In [ ]:
# Rename the columns
dsr = daily_rate.rename(columns = {'FullDate':'ds','Tmax':'add2',
                                           'SysLoad':'add1','GasPrice':'add3',
                                           'ElecPrice':'y'})
dsr.head(3)

In [ ]:
# Split the dataset into train and test sets
train_dsr = dsr.iloc[:2757,:]
test_dsr = dsr.iloc[2757:,:]
# Model implementation
model = Prophet()
model.add_regressor('add1')
model.add_regressor('add2')
model.add_regressor('add3')
model = model.fit(train_dsr)

In [ ]:
forecast_new_result = model.predict(test_dsr)
MAPE = Metric(test_dsr['y'],forecast_new_result['yhat'])
round(MAPE,2)

In [ ]:
RMSE = np.sqrt(metrics.mean_squared_error(test_dsr['y'],forecast_new_result['yhat']))
round(RMSE,2)

In [ ]:
model.plot_components(forecast_new_result)

In [ ]:
# Split the dataset
trainData = daily_rate.iloc[:2757,:]
testData = daily_rate.iloc[2757:,:]
trainData.shape , testData.shape

In [ ]:
# Exclude the Tmax 
train_df = trainData.iloc[:,[0,2,3,4]]
test_df = testData.iloc[:,[0,2,3,4]]
# Exclude the Tmax 
train_df = trainData.iloc[:,[0,2,3,4]]
test_df = testData.iloc[:,[0,2,3,4]]
# Rename the columns
train_df = train_df.rename(columns = {'FullDate':'ds',
                                           'SysLoad':'add1','GasPrice':'add2',
                                           'ElecPrice':'y'})
test_df = test_df.rename(columns = {'FullDate':'ds',
                                           'SysLoad':'add1','GasPrice':'add2',
                                           'ElecPrice':'y'})
train_df.head(2)

In [ ]:
# Model Implementation
model = Prophet()
model.add_regressor('add1')
model.add_regressor('add2')
model = model.fit(train_df)
forecast_values_new = model.predict(test_df)

In [ ]:
MAPE = Metric(test_df['y'],forecast_values_new['yhat'])
round(MAPE,2)

In [ ]:
model.plot_components(forecast_values_new)